In [1]:
import json
import uuid
from cryptography.fernet import Fernet

# Generate a unique decentralized identifier (DID)
def generate_did():
    return f"did:example:{uuid.uuid4()}"

# Encrypt data to simulate cryptographic proof
def encrypt_data(data, key):
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data

# Decrypt data
def decrypt_data(encrypted_data, key):
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data

# Create a verifiable credential
def create_verifiable_credential(student_info, issuer, key):
    credential = {
        "context": "https://www.w3.org/2018/credentials/v1",
        "type": ["VerifiableCredential"],
        "issuer": issuer,
        "credentialSubject": student_info,
    }
    serialized_credential = json.dumps(credential, indent=2)
    encrypted_credential = encrypt_data(serialized_credential, key)
    return encrypted_credential

# Main function to create a DID and VC
def main():
    # Generate cryptographic key
    key = Fernet.generate_key()
    
    # University and student info
    university_did = generate_did()
    student_info = {
        "id": generate_did(),
        "name": "John Doe",
        "studentId": "UNINotts-123456",
        "course": "Computer Science"
    }

    # Create the VC
    encrypted_vc = create_verifiable_credential(student_info, university_did, key)
    print("Encrypted Verifiable Credential:")
    print(encrypted_vc)
    
    # Decrypting the VC for display
    decrypted_vc = decrypt_data(encrypted_vc, key)
    print("\nDecrypted Verifiable Credential:")
    print(decrypted_vc)

if __name__ == "__main__":
    main()


Encrypted Verifiable Credential:
b'gAAAAABmMmxiSI5GwUKJeJf00d1LgIAyK3rDAxq8c1nUi4YW-iiwGv4vsju_NoMCmEoLzze3CfXb26y1jqTHCLWBVMt89HcDX57nDFZqLkhZKAwg4N0mP6QibtUM5D9YmKmDWfQvyDJVDlZi0HYEXU4upvhNRj5BBYAOoM6f0DvzP4GOXh-9w29481qapK9AB7amhSB5s3iOeUtQiDRYrcsPRSAq9OtpE4VEuwdiNrQMUSnoXES7DeAfqkWhwA8EA8PZxB5S8-MVmEnOswdIbEWFjDKRupNJWsSISYv8fTNLZrHjAJWcjU9OHFDJlvFH8_nVszy6PZALXJXoEBy7H1L1pYveqtC9HNqRe5-ajUh_9f0KjI3otnhnrBGLxxX8rdAV91r0eVAnoc2HFMgtxNE1EpATFDpdgVS6GKG0zb3_SEz2UFUA9ApiBwMr_TwCVOmM5pfIHeinRLUkjw-htALSb_ln-SVZ52tomQz2h5b-l9TT5MKuEY6XhLI7YrqoI5x_ax1IBWUupbf3jcA4XAFiARdNv6YbEQ=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:9bc14b2c-4875-494b-bacd-568ad0744362",
  "credentialSubject": {
    "id": "did:example:672594b3-dd95-4860-9c54-454a4eeea0b3",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}


In [2]:
# !pip install qrcode

In [3]:
import json
import uuid
import qrcode
from cryptography.fernet import Fernet

# Generate a unique decentralized identifier (DID)
def generate_did():
    return f"did:example:{uuid.uuid4()}"

# Encrypt data to simulate cryptographic proof
def encrypt_data(data, key):
    f = Fernet(key)
    encrypted_data = f.encrypt(data.encode())
    return encrypted_data

# Decrypt data
def decrypt_data(encrypted_data, key):
    f = Fernet(key)
    decrypted_data = f.decrypt(encrypted_data).decode()
    return decrypted_data

# Create a verifiable credential
def create_verifiable_credential(student_info, issuer, key):
    credential = {
        "context": "https://www.w3.org/2018/credentials/v1",
        "type": ["VerifiableCredential"],
        "issuer": issuer,
        "credentialSubject": student_info,
    }
    serialized_credential = json.dumps(credential, indent=2)
    encrypted_credential = encrypt_data(serialized_credential, key)
    return encrypted_credential

# Generate QR Code for any text
def generate_qr_code(data):
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4,
    )
    qr.add_data(data)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save("did_qr_code.png")  # Saving the QR code to a file
    return "QR code generated and saved as did_qr_code.png"

# Main function to create a DID and VC
def main():
    # Generate cryptographic key
    key = Fernet.generate_key()
    
    # University and student info
    university_did = generate_did()
    student_info = {
        "id": generate_did(),
        "name": "John Doe",
        "studentId": "UNINotts-123456",
        "course": "Computer Science"
    }

    # Create the VC
    encrypted_vc = create_verifiable_credential(student_info, university_did, key)
    print("Encrypted Verifiable Credential:")
    print(encrypted_vc)
    
    # Decrypting the VC for display
    decrypted_vc = decrypt_data(encrypted_vc, key)
    print("\nDecrypted Verifiable Credential:")
    print(decrypted_vc) 

    # Generate QR Code for the DID
    qr_code_message = generate_qr_code(json.dumps(student_info))
    print(qr_code_message)

if __name__ == "__main__":
    main()


Encrypted Verifiable Credential:
b'gAAAAABmMmxkCtRFPkIRs1ejg6pxu2djH6ntTWm_F-KwEFKX-fZxe1qGPfsh-d25emt1_aTFIpDHlVBpIBDM9SGcorGz3-5kjHW8nN2A9DysnsaI63MAUoYjXtS0IFuxMgTMY7Y3Khn8CMDVngnh_DzAwvZTlCf6hQhshUeF0Z_65Voxwor-Ss6pFWnSSA2CQLJ18rHWJ7aEhR5Hyj3aN48e7nd-fQdavqtaKM0-rPAMrs1AO2jPqPn1Uvc0iaUHyGFDjNqhWkjPDP601sjuIJzLuEJTBqtqhQTUQtS0h0D_VUEIB6Fsbrq6Gln5LZ82YpaV1Go4uSmVO-pOueFCfcwZF6h-apJCxd0CqTbU5aQQ_qo1K9cqMYrncGT5CThfBpO_Cc2lqSpT9yxey0CdDqFoM6KPi_EWyPCAh1doWq2LJLWW5E57U6Kisn0IymUyE6Lih_9DxoLVXPqVxvNZVowxSEXljU0dU35lUdiaVe3RZkgDPNYtLC_qkbO9YIxVHSfBDiqpr5_ZxBKHA8UDXUPMq_3kbgVQEQ=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:9ab182e8-687a-434c-b461-54119801c048",
  "credentialSubject": {
    "id": "did:example:0c017f89-63b8-4fc0-929a-35534e939cc3",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}
QR code generated and saved as

In [4]:
# Generate cryptographic key
key = Fernet.generate_key()

# University and student info
university_did = generate_did()
student_info = {
    "id": generate_did(),
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
}

# Create the VC
encrypted_vc = create_verifiable_credential(student_info, university_did, key)
print("Encrypted Verifiable Credential:")
print(encrypted_vc)

# Decrypting the VC for display
decrypted_vc = decrypt_data(encrypted_vc, key)
print("\nDecrypted Verifiable Credential:")
print(decrypted_vc) 

Encrypted Verifiable Credential:
b'gAAAAABmMmxluA15iukT-2fA5wVJGgLacybq8cSrC8okAvzF9c284deBzM35nRED2I_dG74oZFjeaXUiHYVVi8mPPrPkgCMRr9nCV0es2KPJfqhh7l4cZ8IKoxGVxf-AdEnsnm1xZ4AsT_xtlyLW2MMCAm0Ujt20GP59ZtlPBqaKHYqLQpoy3Whl43URD8WjbY0_WphaGMJIw5tYI955SnASnLP62P54aCqNHvquLjCKGi9Bbi-xop76xtDC9oQ7WVW02a7ssLPPNea5og-MqNE6GT9fyRr-y99EQXV1lcmpd51FoEhnyhSDT3Y8Sy3aY1Ri4gNcL8IJv7vnmRPbhVAquahHCmuxE27O2w6oLDXfE0PzpVCGBI2potEgYIfAwP55yYrujwqDJqgnJBce9DPp2eZC9TlAoWFVqOWd7qIX63JBCOxuSh8ML9Ny0vXCSOdFaB7xxQjTBghk8c86QdwogI0K2F7pZd4QZRs3TAFvXGBwxajbUAYY7nnYTTmJIWaIGmAFz3SAlpP7KTvuyKdSfZxzugXGjg=='

Decrypted Verifiable Credential:
{
  "context": "https://www.w3.org/2018/credentials/v1",
  "type": [
    "VerifiableCredential"
  ],
  "issuer": "did:example:1691d8c0-15c6-4142-be1a-522df01281e5",
  "credentialSubject": {
    "id": "did:example:f7258e72-55a4-48db-8742-a95fb6bf3577",
    "name": "John Doe",
    "studentId": "UNINotts-123456",
    "course": "Computer Science"
  }
}


In [5]:
encrypted_vc

b'gAAAAABmMmxluA15iukT-2fA5wVJGgLacybq8cSrC8okAvzF9c284deBzM35nRED2I_dG74oZFjeaXUiHYVVi8mPPrPkgCMRr9nCV0es2KPJfqhh7l4cZ8IKoxGVxf-AdEnsnm1xZ4AsT_xtlyLW2MMCAm0Ujt20GP59ZtlPBqaKHYqLQpoy3Whl43URD8WjbY0_WphaGMJIw5tYI955SnASnLP62P54aCqNHvquLjCKGi9Bbi-xop76xtDC9oQ7WVW02a7ssLPPNea5og-MqNE6GT9fyRr-y99EQXV1lcmpd51FoEhnyhSDT3Y8Sy3aY1Ri4gNcL8IJv7vnmRPbhVAquahHCmuxE27O2w6oLDXfE0PzpVCGBI2potEgYIfAwP55yYrujwqDJqgnJBce9DPp2eZC9TlAoWFVqOWd7qIX63JBCOxuSh8ML9Ny0vXCSOdFaB7xxQjTBghk8c86QdwogI0K2F7pZd4QZRs3TAFvXGBwxajbUAYY7nnYTTmJIWaIGmAFz3SAlpP7KTvuyKdSfZxzugXGjg=='

In [6]:
test = json.dumps(student_info)

In [7]:
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
qr.add_data(encrypted_vc)
qr.make(fit=True)
img = qr.make_image(fill_color="black", back_color="white")
img.save("test/did_qr_code.png")  # Saving the QR code to a file

In [8]:
# Generate QR Code for the DID
qr_code_message = generate_qr_code(encrypted_vc)
print(qr_code_message)

QR code generated and saved as did_qr_code.png


In [9]:
from cryptography.fernet import Fernet

def verify_credential(encrypted_credential, key):
    try:
        f = Fernet(key)
        decrypted_data = f.decrypt(encrypted_credential).decode()
        credential = json.loads(decrypted_data)
        
        # Check for validity, integrity, authenticity
        if verify_signature(credential):
            print("Credential is valid and authentic.")
            return credential
        else:
            print("Invalid signature. Credential could be tampered with.")
    except Exception as e:
        print(f"Error during verification: {str(e)}")

def verify_signature(credential):
    # Dummy function for signature verification
    return True  # In real implementation, you would check the digital signature here

# Example usage:
# Assume `encrypted_credential` is what you retrieved and decrypted from the database after scanning the QR
# `key` is the Fernet key used for decryption
# decrypted_credential = verify_credential(encrypted_credential, key)


### Reading qr code

In [10]:
# !pip install pillow

In [11]:
# !pip install opencv-python

In [12]:
# !pip install pyzbar

In [13]:
import cv2
from pyzbar import pyzbar

In [14]:
def read_barcodes(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        x, y , w, h = barcode.rect
        #1
        barcode_info = barcode.data.decode('utf-8')
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)
        
        #2
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2.0, (255, 255, 255), 1)
        #3
        with open("barcode_result.txt", mode ='w') as file:
            file.write("Recognized Barcode:" + barcode_info)
    return frame

In [19]:
trial_img= cv2.imread('test/did_qr_code.png')

In [20]:
qcd = cv2.QRCodeDetector()

In [22]:
retval, decoded_info, points, straight_qrcode = qcd.detectAndDecodeMulti(trial_img)

In [23]:
retval

True

In [24]:
decoded_info

('',)